<a href="https://colab.research.google.com/github/Sai0472003/PatentLitigation_Impact/blob/main/Data_Cleaning2(FJC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal effect of Patent Litigation Outcomes on Firm Financials


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
JFC_1988_Present = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1988_to_Present.txt', sep='\t', encoding='latin1')

<ipython-input-2-caae5146eea8>:1: DtypeWarning: Columns (1,2,9,10,11,13,19,20,23,24,26,38,39,40,41,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  JFC_1988_Present = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1988_to_Present.txt', sep='\t', encoding='latin1')


In [3]:
cases = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/cases.csv')
cases = cases[['case_row_id', 'case_number', 'case_name', 'case_cause', 'date_filed', 'date_closed', 'jurisdictional_basis']]

In [4]:
#ISOLATING PANTENT AND TRADENARK INFRINGEMENT SUITES, AND ONLY ORIGINAL PROCEEDINGS
Patents_1988_forwards = pd.DataFrame(JFC_1988_Present[JFC_1988_Present['NOS'].isin([830, 840])]) #only patent/trademark cases
#Patents_1988_forwards  = Patents_1988_forwards[Patents_1988_forwards ['ORIGIN'] == 1] #only original filings
#Patents_1988_forwards  = Patents_1988_forwards[Patents_1988_forwards ['ARBIT'].astype(str) == '-8'] #no formal arbitration courts

clean_JFC_patents = Patents_1988_forwards.copy()

#removing columns that aren't relavent or mostly empty
clean_JFC_patents.drop(columns=['ORIGIN','TITL', 'SECTION', 'SUBSECT', 'ARBIT',
                                'FILEJUDG', 'FILEMAG', 'MDLDOCK',  'TRANSDAT',
                                'TRANSOFF', 'TRANSDOC', 'TRANSORG', 'TDATEUSE',
                                'TRCLACT', 'TERMJUDG', 'TERMMAG', 'DJOINED',
                                'PRETRIAL', 'TRIBEGAN', 'TRIALEND', 'AMTREC',
                                'TRMARB', 'RESIDENC', 'PROSE', 'IFP', 'TAPEYEAR'],
                                 inplace=True)

In [5]:
clean_JFC_patents.loc[clean_JFC_patents['CLASSACT'] == -8, 'CLASSACT'] = 0

In [6]:
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['DOCKET'].astype(str).str[:2] + "-cv-" + clean_JFC_patents['DOCKET'].astype(str).str[2:]
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['JURIS'].astype(str) + "-" + clean_JFC_patents['FILEDATE'].astype(str).str.split("/").str[0] + "/" + clean_JFC_patents['FILEDATE'].astype(str).str.split("/").str[1] + "/" + clean_JFC_patents['unique_identifier'].astype(str)
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['unique_identifier'] + "-" + clean_JFC_patents['TERMDATE'].astype(str)

#creating common unique identifier to match JFC dataset
cases['unique_identifier'] = cases['case_number'].astype(str).str.split(":", n=1).str[-1]
cases['unique_identifier'] = cases['date_filed'].astype(str).str.split("-").str[1] + "/" + cases['date_filed'].astype(str).str.split("-").str[2] + "/" +cases['unique_identifier'].astype(str)

for index, row in cases.iterrows():
    if row['jurisdictional_basis'] == "Federal Question":
        cases.loc[index, 'unique_identifier'] = "3-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "U.S. Government Plaintiff":
        cases.loc[index, 'unique_identifier'] = "1-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "U.S. Government Defendant":
        cases.loc[index, 'unique_identifier'] = "2-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "Diversity":
        cases.loc[index, 'unique_identifier'] = "4-" + row['unique_identifier']

cases['unique_identifier'] = cases['unique_identifier'] + "-" + cases['date_closed'].astype(str).str.split("-").str[1] + "/" + cases['date_closed'].astype(str).str.split("-").str[2] + "/" + cases['date_closed'].astype(str).str.split("-").str[0]

In [82]:
clean_JFC_patents.drop_duplicates(subset=['unique_identifier'], inplace=True)
clean_JFC_patents['unique_identifier'].nunique() / len(clean_JFC_patents)

1.0

In [64]:
clean_JFC_patents

,CIRCUIT,DISTRICT,OFFICE,DOCKET,FILEDATE,FDATEUSE,JURIS,NOS,JURY,CLASSACT,...,COUNTY,PLT,DEF,TERMDATE,PROCPROG,DISP,NOJ,JUDGMENT,STATUSCD,unique_identifier
42,0,90,1,8203478,12/08/1982,12/01/1982,3,830,-8,0,...,99999,CHUBB INTEG,NATL BANK WASH,02/10/1988,3,13,-8,-8,NaN,3-12/08/82-cv-03478-02/10/1988
47,0,90,1,8300001,01/03/1983,01/01/1983,3,830,-8,0,...,88888,NEWMAN,MOSSINGHOFF,02/17/1988,8,9,0,2,NaN,3-01/03/83-cv-00001-02/17/1988
188,0,90,1,8403233,10/22/1984,10/01/1984,3,830,-8,0,...,88888,CORPCOMMUNICATIONS,CAPITOL,06/01/1988,2,0,-8,-8,NaN,3-10/22/84-cv-03233-06/01/1988
241,0,90,1,8403720,12/06/1984,12/01/1984,3,840,-8,0,...,11001,WASH CHIROPTC HLTH,CHIROPTC INC,10/01/1987,3,3,-8,-8,NaN,3-12/06/84-cv-03720-10/01/1987
269,0,90,1,8500006,05/22/1985,05/01/1985,3,830,-8,0,...,0,FOUR PILLARS ENTP CO,CHANG ETAL,06/01/1988,4,6,0,2,NaN,3-05/22/85-cv-00006-06/01/1988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10445328,11,3J,4,2100368,12/31/2021,12/01/2021,3,830,P,0,...,13051,FACTORY DIRECT WHOLESALE LLC,OFFICE KICK INC,01/01/1900,-8,-8,-8,-8,S,3-12/31/21-cv-00368-01/01/1900
10445345,11,3J,4,2200291,12/12/2022,12/01/2022,3,840,B,0,...,13051,CRESTWOOD MANAGEMENT L,FERGUSON,01/01/1900,-8,-8,-8,-8,S,3-12/12/22-cv-00291-01/01/1900
10445359,11,3J,4,2300115,05/01/2023,05/01/2023,3,840,B,0,...,13051,VPR BRANDS LP,VAPE LOFT ABERCORN LLC,01/01/1900,-8,-8,-8,-8,S,3-05/01/23-cv-00115-01/01/1900
10445495,11,3J,4,2400155,07/22/2024,07/01/2024,3,840,P,0,...,13051,TACMED SOLUTIONS LLC,ROGUE RANGER LLC,10/23/2024,4,5,0,4,L,3-07/22/24-cv-00155-10/23/2024


In [78]:
cases.groupby('case_number')['unique_identifier'].nunique().value_counts()
#seems to be errors with mulitple filing/closing dates within the df
#this causes there to be more than one unique_identifiers within a given case

,count
unique_identifier,
1,83411
2,5096
0,1607
3,453
4,33
5,3


In [88]:
# Identify case_number values that have exactly 1 unique_identifier
valid_cases = cases.groupby('case_number')['unique_identifier'].nunique()
valid_cases = valid_cases[valid_cases == 1].index  # Keep only case_number values with exactly 1 unique_identifier

# Filter the dataframe to retain only those case_number values
cases = cases[cases['case_number'].isin(valid_cases)]
cases.groupby('case_number')['unique_identifier'].nunique().value_counts()

,count
unique_identifier,
1,83411


In [91]:
cases_JFC = pd.merge(cases, clean_JFC_patents, on='unique_identifier', how='inner')
cases_JFC.groupby('case_number')['unique_identifier'].nunique().value_counts()

,count
unique_identifier,
1,54022


In [106]:
# Identify case_number values that appear more than once
duplicate_cases = cases_JFC['case_number'].value_counts()
duplicate_cases = duplicate_cases[duplicate_cases > 1].index  # Get case_number values that appear more than once

# Drop rows where case_number is in the duplicate list
cases_JFC = cases_JFC[~cases_JFC['case_number'].isin(duplicate_cases)]

In [118]:
cases_JFC.drop_duplicates(subset=['unique_identifier'], inplace=True)

<ipython-input-118-d9526e45a54f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases_JFC.drop_duplicates(subset=['unique_identifier'], inplace=True)


In [123]:
pd.set_option('display.max_columns', None)
cases_JFC.head()

,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,unique_identifier,CIRCUIT,DISTRICT,OFFICE,DOCKET,FILEDATE,FDATEUSE,JURIS,NOS,JURY,CLASSACT,DEMANDED,COUNTY,PLT,DEF,TERMDATE,PROCPROG,DISP,NOJ,JUDGMENT,STATUSCD
0,6908.0,1:98-cv-00596,Mettke v. Touchnet Information,35:183 Patent Infringement,1998-03-16,1998-09-21,Federal Question,3-03/16/98-cv-00596-09/21/1998,11,26,1,9800596,03/16/1998,03/01/1998,3,830,-8,0,0,1015,METTKE,TOUCHNET INFORMATION,09/21/1998,3,14,-8,-8,NaN
1,6933.0,1:98-cv-00654,"Monsanto Co. Inc. v. Styron, et al",28:1338 Patent Infringement,1998-06-29,1999-03-15,Federal Question,3-06/29/98-cv-00654-03/15/1999,11,28,1,9800654,06/29/1998,06/01/1998,3,830,-8,0,0,1097,MONSANTO CO INC,STYRON,03/15/1999,10,13,-8,-8,NaN
2,86730.0,1:18-cv-00012,Sundesa LLC v. Eurark LLC,35:271 Patent Infringement,2018-02-13,2018-07-06,Federal Question,3-02/13/18-cv-00012-07/06/2018,8,60,1,1800012,02/13/2018,02/01/2018,3,830,B,0,0,88888,SUNDESA LLC,EURARK LLC,07/06/2018,3,13,0,0,L
3,30787.0,2:99-cv-00674,"Star Video Duplictns v. US Magnetics Corp, et al",15:1125 Trademark Infringement (Lanham Act),1999-04-09,1999-12-16,Federal Question,3-04/09/99-cv-00674-12/16/1999,9,70,2,9900674,04/09/1999,04/01/1999,3,830,-8,0,0,4013,STAR VIDEO DUPLICTNS,US MAGNETICS CORP,12/16/1999,10,13,-8,-8,NaN
4,30985.0,2:99-cv-02348,"Brother Industries v. Jing Mold Enterprise, et al",28:1338 Patent Infringement,1999-03-05,2001-03-13,Federal Question,3-03/05/99-cv-02348-03/13/2001,9,73,2,9902348,03/05/1999,03/01/1999,3,830,-8,0,0,99999,BROTHER INDUSTRIES,JING MOLD ENTERPRISE,03/13/2001,3,13,-8,-8,L


In [127]:
clean_cases_JFC = cases_JFC.copy()
clean_cases_JFC.drop(columns=['unique_identifier', 'DOCKET', 'FILEDATE',
                               'FDATEUSE', 'TERMDATE', 'STATUSCD'])

,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,CIRCUIT,DISTRICT,OFFICE,JURIS,NOS,JURY,CLASSACT,DEMANDED,COUNTY,PLT,DEF,PROCPROG,DISP,NOJ,JUDGMENT
0,6908.0,1:98-cv-00596,Mettke v. Touchnet Information,35:183 Patent Infringement,1998-03-16,1998-09-21,Federal Question,11,26,1,3,830,-8,0,0,1015,METTKE,TOUCHNET INFORMATION,3,14,-8,-8
1,6933.0,1:98-cv-00654,"Monsanto Co. Inc. v. Styron, et al",28:1338 Patent Infringement,1998-06-29,1999-03-15,Federal Question,11,28,1,3,830,-8,0,0,1097,MONSANTO CO INC,STYRON,10,13,-8,-8
2,86730.0,1:18-cv-00012,Sundesa LLC v. Eurark LLC,35:271 Patent Infringement,2018-02-13,2018-07-06,Federal Question,8,60,1,3,830,B,0,0,88888,SUNDESA LLC,EURARK LLC,3,13,0,0
3,30787.0,2:99-cv-00674,"Star Video Duplictns v. US Magnetics Corp, et al",15:1125 Trademark Infringement (Lanham Act),1999-04-09,1999-12-16,Federal Question,9,70,2,3,830,-8,0,0,4013,STAR VIDEO DUPLICTNS,US MAGNETICS CORP,10,13,-8,-8
4,30985.0,2:99-cv-02348,"Brother Industries v. Jing Mold Enterprise, et al",28:1338 Patent Infringement,1999-03-05,2001-03-13,Federal Question,9,73,2,3,830,-8,0,0,99999,BROTHER INDUSTRIES,JING MOLD ENTERPRISE,3,13,-8,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54028,30718.0,2:99-cv-00216,"Impact Roller Tech, et al v. Star Aggregates Inc",28:1338 Patent Infringement,1999-10-06,2000-03-28,Federal Question,10,89,2,3,830,-8,0,0,88888,IMPACT ROLLER TECH,STAR AGGREGATES INC,5,5,0,1
54029,28107.0,2:93-cv-00004,Construction Tech v. Heating,35:271 Patent Infringement,1993-01-04,1993-08-11,Federal Question,10,89,2,3,830,-8,0,0,88888,CONSTRUCTION TECH,HEATING,3,12,-8,-8
54030,28883.0,2:95-cv-00217,Wyoming Woolens v. Acorn Products Co,28:1338 Patent Infringement,1995-09-29,1995-10-06,Federal Question,10,89,2,3,830,-8,0,0,56039,WYOMING WOOLENS,ACORN PRODUCTS CO,1,12,-8,-8
54031,29713.0,2:97-cv-00019,Chey Land & Cattle v. Crown Cork & Seal Co,35:271 Patent Infringement,1997-01-24,1997-07-25,Federal Question,10,89,2,3,830,-8,0,2000,56021,CHEY LAND & CATTLE,CROWN CORK & SEAL CO,1,12,-8,-8


# ***Querying and Merging Data on Company Financials from WRDS***

In [128]:
!pip install wrds
import wrds
db = wrds.Connection(wrds_username="sai047")

Loading library list...
Done


In [129]:
query = """
SELECT gvkey, conm
FROM comp.company
"""
comp_names = db.raw_sql(query)

In [130]:
comp_names

,gvkey,conm
0,001000,A & E PLASTIK PAK INC
1,001001,A & M FOOD SERVICES INC
2,001002,AAI CORP
3,001003,A.A. IMPORTING CO INC
4,001004,AAR CORP
...,...,...
55007,356128,JOINT STOCK COMPANY KASPI KZ
55008,356289,SOLARBANK CORP
55009,356687,ATHA ENERGY CORP
55010,356859,ACUREN CORP


In [131]:
gvkey_plt = pd.DataFrame(clean_cases_JFC[['PLT']])
gvkey_def = pd.DataFrame(clean_cases_JFC[['DEF']])

In [133]:
def clean_names(name):
    name = name.replace(",", '')  # Remove Commas
    name = name.replace(".", "")  # Remove periods
    name = name.replace("ET AL", "").strip()  # Remove 'ET AL' (case insensitive if already upper)
    return name

gvkey_def = pd.DataFrame(gvkey_def['DEF'].apply(clean_names)) # Apply to the 'DEF' column
gvkey_plt = pd.DataFrame(gvkey_plt['PLT'].apply(clean_names)) # Apply to the 'PLT' column
comp_names['conm'].apply(clean_names)
clean_cases_JFC['PLT'] = clean_cases_JFC['PLT'].apply(clean_names)
clean_cases_JFC['DEF'] = clean_cases_JFC['DEF'].apply(clean_names)

In [134]:
comp_names['name'] = comp_names['conm']
comp_names.drop(columns=['conm'], inplace=True)
gvkey_plt['name'] = gvkey_plt['PLT']
gvkey_def['name'] = gvkey_def['DEF']

In [135]:
gvkey_def = gvkey_def.merge(comp_names, on='name', how='inner')
gvkey_def.drop(columns=['name'], inplace=True)

gvkey_plt = gvkey_plt.merge(comp_names, on='name', how='inner')
gvkey_plt.drop(columns=['name'], inplace=True)

In [136]:
gvkey_def

,DEF,gvkey
0,INTERMEDICS INC,006045
1,TEXAS INSTRUMENTS INC,010499
2,ENCORE COMPUTER CORP,004344
3,EBAY INC,114524
4,LEGGETT & PLATT INC,006649
...,...,...
5921,OEC MEDICAL SYSTEMS INC,003935
5922,CRAY INC,061350
5923,CISCO SYSTEMS INC,020779
5924,HUBBELL INC,005764


In [137]:
gvkey_plt

,PLT,gvkey
0,KIDDE INC,006430
1,SONOCO PRODUCTS CO,009815
2,DCL INC,003688
3,INTERGRAPH CORP,006036
4,ALABAMA POWER CO,001225
...,...,...
2610,GILEAD SCIENCES INC,024856
2611,PFIZER INC,008530
2612,JAPAN TOBACCO INC,223528
2613,PFIZER INC,008530


In [138]:
plaintiff_RnD = pd.merge(clean_cases_JFC[['case_row_id', 'case_number', 'date_filed',
                                 'date_closed', 'PLT']], gvkey_plt, on='PLT',
                                  how='inner')
plaintiff_RnD['case_number'].nunique()
plaintiff_RnD.drop_duplicates(inplace=True)

In [140]:
plaintiff_RnD

,case_row_id,case_number,date_filed,date_closed,PLT,gvkey
0,58881.0,4:85-cv-00399,1985-04-25,1989-10-23,KIDDE INC,006430
1,59800.0,4:96-cv-02682,1996-09-03,1999-02-23,SONOCO PRODUCTS CO,009815
6,7660.0,1:99-cv-01028,1999-04-26,1999-06-08,DCL INC,003688
7,64094.0,5:97-cv-03023,1997-11-17,2002-05-06,INTERGRAPH CORP,006036
8,30252.0,2:98-cv-00323,1998-02-11,1998-06-10,ALABAMA POWER CO,001225
...,...,...,...,...,...,...
52945,25530.0,1:15-cv-00149,2015-08-31,2015-11-02,GILEAD SCIENCES INC,024856
52970,98842.0,1:19-cv-00097,2019-04-29,2019-10-03,PFIZER INC,008530
53042,98446.0,1:19-cv-00012,2019-01-24,2019-05-09,JAPAN TOBACCO INC,223528
53044,91470.0,1:20-cv-00244,2020-10-16,2021-06-09,PFIZER INC,008530


In [142]:
defendent_RnD = pd.merge(clean_cases_JFC[['case_row_id', 'case_number', 'date_filed',
                                 'date_closed', 'DEF']], gvkey_def, on='DEF',
                                  how='inner')

defendent_RnD['case_number'].nunique()
defendent_RnD.drop_duplicates(inplace=True)

In [143]:
defendent_RnD

,case_row_id,case_number,date_filed,date_closed,DEF,gvkey
0,59316.0,4:93-cv-00626,1993-06-25,1996-10-08,INTERMEDICS INC,006045
5,78121.0,2:11-cv-01037,2011-12-06,2015-03-30,TEXAS INSTRUMENTS INC,010499
25,78037.0,5:92-cv-02573,1992-11-02,1996-07-29,ENCORE COMPUTER CORP,004344
26,26100.0,1:15-cv-01059,2015-06-24,2017-05-15,EBAY INC,114524
68,66037.0,5:11-cv-01349,2011-04-20,2012-09-17,LEGGETT & PLATT INC,006649
...,...,...,...,...,...,...
196991,50230.0,3:97-cv-00860,1997-12-18,1998-03-12,OEC MEDICAL SYSTEMS INC,003935
196992,90118.0,3:18-cv-00318,2018-05-02,2019-04-24,CRAY INC,061350
196996,50940.0,3:99-cv-00782,1999-12-10,2000-03-23,CISCO SYSTEMS INC,020779
197094,67177.0,6:99-cv-00342,1999-05-03,2001-01-19,HUBBELL INC,005764


In [146]:
set1 = set(defendent_RnD['case_number'].unique())
set2 = set(plaintiff_RnD['case_number'].unique())
common_elements = set1.intersection(set2)

print("The number of unique cases with known gvkeys:",
      len(set1) + len(set2) - len(common_elements))

The number of unique cases with known gvkeys: 8248


In [ ]:
# Convert all relevant date columns to datetime format
plaintiff_RnD["date_filed"] = pd.to_datetime(plaintiff_RnD["date_filed"])
plaintiff_RnD["date_closed"] = pd.to_datetime(plaintiff_RnD["date_closed"])
defendent_RnD["date_filed"] = pd.to_datetime(defendent_RnD["date_filed"])
defendent_RnD["date_closed"] = pd.to_datetime(defendent_RnD["date_closed"])

In [ ]:
# Get all GVKEYs involved in lawsuits
gvkeys = list(set(list(plaintiff_RnD["gvkey"]) + list(defendent_RnD["gvkey"])))

from pandas.tseries.offsets import DateOffset

# Extend date range by 4 quarters (1 year)
min_date = min(plaintiff_RnD["date_filed"].min(), defendent_RnD["date_filed"].min()) - DateOffset(years=1)
max_date = max(plaintiff_RnD["date_closed"].max(), defendent_RnD["date_closed"].max()) + DateOffset(years=1)

print(f"Querying R&D data from {min_date.date()} to {max_date.date()}")
print(f"Querying financials for {len(gvkeys)} firms from {min_date} to {max_date}")

Querying R&D data from 1975-04-26 to 2023-10-27
Querying financials for 2156 firms from 1975-04-26 00:00:00 to 2023-10-27 00:00:00


In [ ]:
db.describe_table(library="wrdsapps_finratio", table="firm_ratio")

Approximately 2795250 rows in wrdsapps_finratio.firm_ratio.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),Global Company Key
1,permno,True,INTEGER,PERMNO
2,adate,True,DATE,Fiscal year end
3,qdate,True,DATE,Fiscal quarter end
4,public_date,True,DATE,Date
...,...,...,...,...
93,ffi48,True,DOUBLE PRECISION,Fama and French 48 Industries
94,ffi49_desc,True,VARCHAR(5),Fama and French 49 Industries - Description
95,ffi49,True,DOUBLE PRECISION,Fama and French 49 Industries
96,ticker,True,VARCHAR(8),Ticker Symbol


In [ ]:
query = f"""
SELECT gvkey, qdate, rd_sale
FROM wrdsapps_finratio.firm_ratio
WHERE gvkey IN ({",".join("'" + str(g) + "'" for g in gvkeys)})
AND qdate BETWEEN '{min_date}' AND '{max_date}'
AND rd_sale IS NOT NULL
"""
rd_sale_data = db.raw_sql(query)

In [ ]:
rd_sale_data

,gvkey,qdate,rd_sale
0,001013,1982-01-31,0.079463
1,001013,1982-01-31,0.079463
2,001013,1982-01-31,0.079463
3,001013,1982-04-30,0.079463
4,001013,1982-04-30,0.079463
...,...,...,...
378825,347007,2023-06-30,442.771959
378826,347007,2023-06-30,442.771959
378827,347007,2023-06-30,442.771959
378828,347007,2023-09-30,429.696043


In [ ]:
# Convert date_filed and date_closed to the last day of the quarter
plaintiff_RnD["filing_quarter"] = plaintiff_RnD["date_filed"] + pd.offsets.QuarterEnd(0)
plaintiff_RnD["closing_quarter"] = plaintiff_RnD["date_closed"] + pd.offsets.QuarterEnd(0)

defendent_RnD["filing_quarter"] = defendent_RnD["date_filed"] + pd.offsets.QuarterEnd(0)
defendent_RnD["closing_quarter"] = defendent_RnD["date_closed"] + pd.offsets.QuarterEnd(0)

In [ ]:
for i in range(1, 4):
    plaintiff_RnD[f"q{i}_before_filing"] = plaintiff_RnD["filing_quarter"] - pd.DateOffset(months=3 * i)
    plaintiff_RnD[f"q{i}_after_closing"] = plaintiff_RnD["closing_quarter"] + pd.DateOffset(months=3 * i)

    defendent_RnD[f"q{i}_before_filing"] = defendent_RnD["filing_quarter"] - pd.DateOffset(months=3 * i)
    defendent_RnD[f"q{i}_after_closing"] = defendent_RnD["closing_quarter"] + pd.DateOffset(months=3 * i)


In [ ]:
defendent_RnD

,case_row_id,case_number,date_filed,date_closed,DEF,gvkey,filing_quarter,closing_quarter,q1_before_filing,q1_after_closing,q2_before_filing,q2_after_closing,q3_before_filing,q3_after_closing
0,59316.0,4:93-cv-00626,1993-06-25,1996-10-08,INTERMEDICS INC,006045,1993-06-30,1996-12-31,1993-03-30,1997-03-31,1992-12-30,1997-06-30,1992-09-30,1997-09-30
5,78121.0,2:11-cv-01037,2011-12-06,2015-03-30,TEXAS INSTRUMENTS INC,010499,2011-12-31,2015-03-31,2011-09-30,2015-06-30,2011-06-30,2015-09-30,2011-03-31,2015-12-31
25,78037.0,5:92-cv-02573,1992-11-02,1996-07-29,ENCORE COMPUTER CORP,004344,1992-12-31,1996-09-30,1992-09-30,1996-12-30,1992-06-30,1997-03-30,1992-03-31,1997-06-30
26,26100.0,1:15-cv-01059,2015-06-24,2017-05-15,EBAY INC,114524,2015-06-30,2017-06-30,2015-03-30,2017-09-30,2014-12-30,2017-12-30,2014-09-30,2018-03-30
69,66037.0,5:11-cv-01349,2011-04-20,2012-09-17,LEGGETT & PLATT INC,006649,2011-06-30,2012-09-30,2011-03-30,2012-12-30,2010-12-30,2013-03-30,2010-09-30,2013-06-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207349,50230.0,3:97-cv-00860,1997-12-18,1998-03-12,OEC MEDICAL SYSTEMS INC,003935,1997-12-31,1998-03-31,1997-09-30,1998-06-30,1997-06-30,1998-09-30,1997-03-31,1998-12-31
207350,90118.0,3:18-cv-00318,2018-05-02,2019-04-24,CRAY INC,061350,2018-06-30,2019-06-30,2018-03-30,2019-09-30,2017-12-30,2019-12-30,2017-09-30,2020-03-30
207354,50940.0,3:99-cv-00782,1999-12-10,2000-03-23,CISCO SYSTEMS INC,020779,1999-12-31,2000-03-31,1999-09-30,2000-06-30,1999-06-30,2000-09-30,1999-03-31,2000-12-31
207454,67177.0,6:99-cv-00342,1999-05-03,2001-01-19,HUBBELL INC,005764,1999-06-30,2001-03-31,1999-03-30,2001-06-30,1998-12-30,2001-09-30,1998-09-30,2001-12-31


In [ ]:
# Ensure qdate is in datetime format
rd_sale_data["qdate"] = pd.to_datetime(rd_sale_data["qdate"])

# Merge R&D ratios for the quarter of case filing & closing
plaintiff_RnD = plaintiff_RnD.merge(
    rd_sale_data, left_on=["gvkey", "filing_quarter"], right_on=["gvkey", "qdate"], how="left"
).rename(columns={"rd_sale": "filing_rd_sale"}).drop(columns=["qdate"])

plaintiff_RnD = plaintiff_RnD.merge(
    rd_sale_data, left_on=["gvkey", "closing_quarter"], right_on=["gvkey", "qdate"], how="left"
).rename(columns={"rd_sale": "closing_rd_sale"}).drop(columns=["qdate"])

defendent_RnD = defendent_RnD.merge(
    rd_sale_data, left_on=["gvkey", "filing_quarter"], right_on=["gvkey", "qdate"], how="left"
).rename(columns={"rd_sale": "filing_rd_sale"}).drop(columns=["qdate"])

defendent_RnD = defendent_RnD.merge(
    rd_sale_data, left_on=["gvkey", "closing_quarter"], right_on=["gvkey", "qdate"], how="left"
).rename(columns={"rd_sale": "closing_rd_sale"}).drop(columns=["qdate"])

In [ ]:
for i in range(1, 4):  # 3 quarters before & after
    plaintiff_RnD = plaintiff_RnD.merge(
        rd_sale_data, left_on=["gvkey", f"q{i}_before_filing"], right_on=["gvkey", "qdate"], how="left"
    ).rename(columns={"rd_sale": f"q{i}_before"}).drop(columns=["qdate"])

    plaintiff_RnD = plaintiff_RnD.merge(
        rd_sale_data, left_on=["gvkey", f"q{i}_after_closing"], right_on=["gvkey", "qdate"], how="left"
    ).rename(columns={"rd_sale": f"q{i}_after"}).drop(columns=["qdate"])

    defendent_RnD = defendent_RnD.merge(
        rd_sale_data, left_on=["gvkey", f"q{i}_before_filing"], right_on=["gvkey", "qdate"], how="left"
    ).rename(columns={"rd_sale": f"q{i}_before"}).drop(columns=["qdate"])

    defendent_RnD = defendent_RnD.merge(
        rd_sale_data, left_on=["gvkey", f"q{i}_after_closing"], right_on=["gvkey", "qdate"], how="left"
    ).rename(columns={"rd_sale": f"q{i}_after"}).drop(columns=["qdate"])

In [ ]:
plaintiff_RnD.drop_duplicates(inplace=True)
defendent_RnD.drop_duplicates(inplace=True)

In [ ]:
rd_sale_data['rd_sale'].isna().sum()

0

In [ ]:
# Ensure filing_quarter & closing_quarter are in datetime format
plaintiff_RnD["filing_quarter"] = pd.to_datetime(plaintiff_RnD["filing_quarter"])
plaintiff_RnD["closing_quarter"] = pd.to_datetime(plaintiff_RnD["closing_quarter"])
defendent_RnD["filing_quarter"] = pd.to_datetime(defendent_RnD["filing_quarter"])
defendent_RnD["closing_quarter"] = pd.to_datetime(defendent_RnD["closing_quarter"])

# Find unique case_numbers that have a match in rd_sale_data for plaintiffs
plaintiff_matches = plaintiff_RnD.merge(
    rd_sale_data, left_on=["gvkey", "filing_quarter"], right_on=["gvkey", "qdate"], how="inner"
).drop_duplicates(subset=["case_number"])["case_number"].nunique()

plaintiff_matches += plaintiff_RnD.merge(
    rd_sale_data, left_on=["gvkey", "closing_quarter"], right_on=["gvkey", "qdate"], how="inner"
).drop_duplicates(subset=["case_number"])["case_number"].nunique()

# Find unique case_numbers that have a match in rd_sale_data for defendants
defendent_matches = defendent_RnD.merge(
    rd_sale_data, left_on=["gvkey", "filing_quarter"], right_on=["gvkey", "qdate"], how="inner"
).drop_duplicates(subset=["case_number"])["case_number"].nunique()

defendent_matches += defendent_RnD.merge(
    rd_sale_data, left_on=["gvkey", "closing_quarter"], right_on=["gvkey", "qdate"], how="inner"
).drop_duplicates(subset=["case_number"])["case_number"].nunique()

# Print results
print(f"Unique plaintiff cases with R&D match: {plaintiff_matches}")
print(f"Unique defendant cases with R&D match: {defendent_matches}")


Unique plaintiff cases with R&D match: 2782
Unique defendant cases with R&D match: 6432


In [ ]:
same_quarter_cases_plaintiff = (plaintiff_RnD["filing_quarter"] == plaintiff_RnD["closing_quarter"]).sum()
same_quarter_cases_defendent = (defendent_RnD["filing_quarter"] == defendent_RnD["closing_quarter"]).sum()

print(f"Plaintiff cases where filing & closing are in the same quarter: {same_quarter_cases_plaintiff}")
print(f"Defendant cases where filing & closing are in the same quarter: {same_quarter_cases_defendent}")

Plaintiff cases where filing & closing are in the same quarter: 159
Defendant cases where filing & closing are in the same quarter: 446
